<a href="https://colab.research.google.com/github/Chthanh/Recommender-System/blob/main/Naive%20Bayes%20Collaborative%20Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Naive Bayes:**
- Các biến cố độc lập $P(A \cap B) = P(A) \cdot P(B)$
- Định lý Bayes: $P(A|B) = \frac{P(A) \cdot P(B|A)}{P(B)}$

**Posterior $\propto$ (Prior $\cdot$ Likelihood)**

# Naive Bayes CF

Sử dụng mô hình Naive Bayes để phân loại rating của user mục tiêu cho item i.

Có 2 cách tiếp cận:

- User-based Naive Bayes CF
- Item-based Naive Bayes CF

## User-based Naive Bayes CF

**Bước 1:** tính xác suất mà user $u$ sẽ đánh giá item $i$ dựa trên các đánh giá của user $u$ trong quá khứ, nghĩa là ta cần tính xác suất
$$P(r_{u,i} = y|\text{Các rating trong tập }I_u),$$
với $I_u$ là tập các item mà user $u$ đã đánh giá.

Theo định lý Bayes:

\begin{equation}
\begin{aligned}
P(r_{u,i} = y|\text{Các rating trong tập }I_u) &= \frac{P(r_i = y) \cdot P(\text{Các rating trong tập }I_u|r_{i} = y)}{P(\text{Các rating trong tập }I_u)} \\
&\propto P(r_i = y)\cdot P(\text{Các rating trong tập }I_u|r_i = y)\\
&\propto P(r_i = y)\cdot \prod_{j \in I_u} P(r_j = r_{u,j}|r_{i} = y)
\end{aligned}
\end{equation}

với $y$ là các giá trị rating có trong tập dữ liệu (ví dụ: -1: không thích, 1: thích).

- $P(r_i = y)$ (prior) là xác suất mà item $i$ được đánh giá là $y$ bởi các user trong tập dữ liệu.

$$P(r_i = y) = \frac{\#\{u \in U| r_{u,i} = y \}+ \alpha}{\#\{u \in U| r_{u,i} \neq \text{NaN}\} + \#R \cdot \alpha}$$

trong đó: - $U$ là tập các user trong tập dữ liệu. - $R$ là các giá trị rating có thể xảy ra (Vd: {−1, 1}). Và $\alpha$ là một giá trị nhỏ được thêm vào để làm thay đổi xác suất đầu ra, tránh hiện tượng xác suất = 0 hoặc không xác định.

- $P(r_j = k|r_i = y)$ (likelihood) là xác suất mà item $j$ được đánh giá = $k$, cho trước rating của item $i$ là $y$ (XS có điều kiện $P(A|B) = \frac{P(A \cap B)}{P(B)}$).
$$P(r_j = k|r_i = y) = \frac{\#\{u \in U| r_{u,j} = k \wedge r_{u,i} = y \}+ \alpha}{\#\{u \in U| r_{u,j} \neq \text{NaN} \wedge r_{u,i} = y\} + \#R \cdot \alpha}$$

**Bước 2:** Ước lượng rating:
- Lấy giá trị $y$ có xác suất hậu nghiệm tính được ở bước 1 cao nhất.
$$\hat r_{u,i} = \text{argmax}_{y} P(r_{u,i} = y|\text{Các rating trong tập }I_u)$$


### Ví dụ 1:

Ta có ma trận rating nhị phân gồm 5 users (dòng) và 6 items (cột) như sau:

ItemId->|i1|i2|i3|i4|i5|i6|
-|:-:|:-:|:-:|:-:|:-:|:-:|
u1|1|-1|1|-1|1|-1|
u2|1|1|?|-1|-1|-1|
u3|?|1|1|-1|-1|?|
u4|-1|-1|-1|1|1|1|
u5|-1|?|-1|1|1|1|

Yêu cầu: dự đoán rating ở các vị trí có dấu `?` của user 2 bằng phương pháp User-based Naive Bayes, smoothing=0.


#### Các bước thực hiện:

- Bước 1: Xác định các giá trị rating có thể có của tập dữ liệu
  - $R = \{-1, 1\}$
- Bước 2: Xác định $I_{u2}$ và item cần dự đoán
  - $I_{u2} =$ {i1, i2, i4, i5, i6}
  - item cần dự đoán rating = {i3}
- Bước 3: Xấp xỉ các xác suất
  \begin{equation}\begin{aligned} P(r_{u2, i3} = 1|r_{i1},r_{i2},r_{i4},r_{i5},r_{i6}) &\propto P(r_{i3} = 1) \cdot \prod_{j \in I_{u2}} P(r_{j} = r_{u2,j}|r_{i3}=1) \\
  & = (0.5) \cdot (1)(0.5)(1)(0.5)(1) = 0.125  
  \end{aligned} \end{equation}

$P(r_{i3} = 1) = \frac{\#\{u1,u3\}}{\#\{u1,u3,u4,u5\}} = \frac{2}{4} = 0.5$

$P(r_{i1}=r_{u2,i1}=1|r_{i3}=1)= \frac{\#\{u1\}}{\#\{u1\}} = \frac{1}{1} = 1$

$P(r_{i2}=r_{u2,i2}=1|r_{i3}=1)=  \frac{\#\{u3\}}{\#\{u1,u3\}} = \frac{1}{2} = 0.5$

$P(r_{i4}=r_{u2,i4}=-1|r_{i3}=1)= \frac{\#\{u1,u3\}}{\#\{u1,u3\}} = \frac{2}{2} = 1$

$P(r_{i5}=-1|r_{i3}=1)= \frac{1}{2} = 0.5$

$P(r_{i6}=-1|r_{i3}=1)= \frac{1}{1} = 1$

$P(r_{u2,i3} = -1|r_{i1},r_{i2},r_{i4},r_{i5},r_{i6}) \propto P(r_{i3} = -1) \cdot \prod_{j \in I_{u2}} P(r_{j}=r_{u2,j}|r_{i3}=-1) = 0$
- Bước 4: Dự đoán rating
  - $P(r_{u2,i3} = 1|\cdot) > P(r_{u2,i3} = -1|\cdot)$ nên $\hat r_{u2,i3} = 1$



#### Sử dụng Python

In [ ]:
# import libraries
import numpy as np
import pandas as pd

In [ ]:
# Ma trận rating
example_1 = np.array([[1,-1,1,-1,1,-1],
              [1,1,np.nan,-1,-1,-1],
              [np.nan,1,1,-1,-1,np.nan],
              [-1,-1,-1,1,1,1],
              [-1,np.nan,-1,1,1,1]])

# Chuyển ma trận A thành dataframe
data_1 = pd.DataFrame(example_1, columns=['i1', 'i2', 'i3', 'i4', 'i5', 'i6'], index=['u1','u2','u3','u4','u5' ])
data_1

,i1,i2,i3,i4,i5,i6
u1,1.0,-1.0,1.0,-1.0,1.0,-1.0
u2,1.0,1.0,NaN,-1.0,-1.0,-1.0
u3,NaN,1.0,1.0,-1.0,-1.0,NaN
u4,-1.0,-1.0,-1.0,1.0,1.0,1.0
u5,-1.0,NaN,-1.0,1.0,1.0,1.0


In [ ]:
def get_labels(df):
  """Hàm trả về các label trong tập dữ liệu"""
  labels = np.unique(df)
  return labels[~np.isnan(labels)]

def get_users(df, picked_item):
  """
  Lọc ra tập user mà picked_item đã được đánh giá và tập user mà picked_item chưa được đánh giá
  @param  df: ma trận URM
          picked_item: item mục tiêu
  @return: 2 dataframe: I_u, predicted_items
  """
  I_u = df[picked_item].dropna(axis=0, how='all')
  predicted_users = df.drop(I_u.index, axis=0, errors='ignore')

  return (I_u, predicted_users)

In [ ]:
labels = get_labels(data_1)
labels

array([-1.,  1.])

In [ ]:
picked_item = 'i3'
I_u, predicted_users = get_users(data_1, picked_item)
print(f"Tập các user đã đánh giá {picked_item} : {I_u.index.values} \n")
print(f"Tập các user chưa đánh giá {picked_item}: {predicted_users.index.values}")

display(predicted_users)


Tập các user đã đánh giá i3 : ['u1' 'u3' 'u4' 'u5'] 

Tập các user chưa đánh giá i3: ['u2']


,i1,i2,i3,i4,i5,i6
u2,1.0,1.0,NaN,-1.0,-1.0,-1.0


In [ ]:
# Hàm tiên nghiệm
def get_prior(predicted_users, labels, smoothing=0):
  """
  Tính xác suất tiên nghiệm của từng label ứng với từng user chưa đánh giá item mục tiêu dựa trên dữ liệu quan sát được
  @param predicted_users: tập dữ liệu gồm các user chưa được đánh giá item mục tiêu
  @return: dictionary, với label là key, xác suất tiên nghiệm là value
  """
  # Khởi tạo dataframe với MultiIndex là (label, predicted_user), column là XS tiên nghiệm ứng với index
  multi_ind = pd.MultiIndex.from_product([labels, predicted_users.index],
                           names=['label', 'item'])
  prior_df = pd.DataFrame(index=multi_ind, columns=['prior'])

  # đếm số lượng rating của từng label
  for i in predicted_users.index:
    prior = {l: predicted_users[predicted_users == l].count().sum() for l in labels}
    total_count = sum(prior.values()) # tổng số lượng rating
    #print(prior)
    for label in prior:
      prior_df.loc[(label,i)]['prior'] = (prior[label]+smoothing) / (total_count + len(labels)*smoothing)
  return prior_df

In [ ]:
smoothing = 0
prior = get_prior(predicted_users, labels, smoothing)
prior

,,prior
label,item,
-1.0,u2,0.6
1.0,u2,0.4


In [ ]:
def get_likelihood(df, I_u, predicted_users, picked_item, labels, smoothing=0):
    """
    Tính xác suất có điều kiện (likelihood)
    @param df: tập dữ liệu
    @param labels: grouped sample indices by class
    @param smoothing: integer, additive smoothing parameter
    @return: dictionary, với label là key, value là xác suất có điều kiện P(feature|class)
    """
    # Khởi tạo dataframe với MultiIndex là (label, predicted_item), columns là các item mà picked_userId đã đánh giá
    multi_ind = pd.MultiIndex.from_product([labels, predicted_users.index],
                           names=['label', 'item'])
    likelihood_df = pd.DataFrame(index=multi_ind, columns=I_u.index)

    # Tính likelihood của từng item trong I_u cho trước rating r_{ui} = label
    for i in predicted_users.index:
      for label in labels:
        for j in I_u.index:
          r_j = df.loc[j,picked_item]
          # Đếm số user mà rating của item i = label và rating của item j = r_j
          count_true = (df.loc[i] == label ) & (df.loc[j]==r_j)
          nominator = count_true.sum()
          # Đếm số user mà rating của item i = label và rating của item j không NaN
          count_truee = (df.loc[i] == label)&(df.loc[j].notna())
          denominator = count_truee.sum()
          # Tính likelihood
          likelihood_df.loc[(label,i)][j] = (nominator+smoothing)/(denominator+len(labels)*smoothing)

    return likelihood_df
likelihood = get_likelihood(data_1, I_u, predicted_users, picked_item, labels, smoothing)
likelihood

NameError: ignored

In [ ]:
def get_posterior(prior, likelihood):
    """
    Tính XS hậu nghiệm dựa trên XS tiên nghiệm và XS hợp lý
    @param prior: dataframe, index là label, column là xác suất tiên nghiệm tương ứng với label
    @param likelihood: dataframe, index là label, column là các XS có điều kiện của các items mà user đã đánh giá
    @return: dataframe, index là label, column là XS hậu nghiệm
    """

    temp_df = pd.merge(prior, likelihood, left_index=True, right_index=True)
    temp_df['posterior'] = temp_df.apply(np.prod, axis=1)
    #display(temp_df)

    return temp_df[['posterior']].reset_index('item').sort_values(by=['item','posterior'], ascending=[True, False])

In [ ]:
posterior = get_posterior(prior, likelihood)
posterior

,item,posterior
label,,
1.0,u2,0.2
-1.0,u2,0.0


In [ ]:
def rating_prediction(posterior):

  score = {}
  for i in posterior['item'].unique():
    score[i] = posterior[posterior['item'] == i].nlargest(1, 'posterior').index.values[0]

  return pd.DataFrame(score.items(), columns=['item', 'pred_label'])

In [ ]:
rating_prediction(posterior)

,item,pred_label
0,u2,1.0


### Ví dụ 2

Dự đoán rating bị khuyết của user u1

In [ ]:
example_2 = np.array([[np.nan,1,2,2,5,np.nan,4,3,5],
              [1,5,3,np.nan,2,3,4,3,np.nan],
              [1,1,2,np.nan,2,4,4,5,np.nan],
              [3,2,2,3,np.nan,1,3,2,np.nan],
              [5,1,5,5,4,4,5,2,np.nan]])

# Chuyển ma trận A thành dataframe
data_2 = pd.DataFrame(example_2, columns=['i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7', 'i8', 'i9'], index=['u1','u2','u3','u4','u5' ])
data_2

,i1,i2,i3,i4,i5,i6,i7,i8,i9
u1,NaN,1.0,2.0,2.0,5.0,NaN,4.0,3.0,5.0
u2,1.0,5.0,3.0,NaN,2.0,3.0,4.0,3.0,NaN
u3,1.0,1.0,2.0,NaN,2.0,4.0,4.0,5.0,NaN
u4,3.0,2.0,2.0,3.0,NaN,1.0,3.0,2.0,NaN
u5,5.0,1.0,5.0,5.0,4.0,4.0,5.0,2.0,NaN


In [ ]:
picked_item = 'i1'
smoothing = 0.01
labels = get_labels(data_2)
I_u, predicted_users = get_users(data_2,picked_item)
prior_df = get_prior(predicted_users, labels, smoothing)
likelihood_df = get_likelihood(data_2, I_u, predicted_users, picked_item, labels, smoothing)
pos_df = get_posterior(prior_df, likelihood_df)
print(f"Items ma user {picked_item} chưa đánh giá: {predicted_users.index.values} \n")
display(prior_df)
display(likelihood_df)
display(pos_df)

Items ma user i1 chưa đánh giá: ['u1'] 



,,prior
label,item,
1.0,u1,0.143262
2.0,u1,0.285106
3.0,u1,0.143262
4.0,u1,0.143262
5.0,u1,0.285106


,,u2,u3,u4,u5
label,item,,,,
1.0,u1,0.009524,0.961905,0.009524,0.009524
2.0,u1,0.009524,0.009524,0.492683,0.980488
3.0,u1,0.009524,0.009524,0.009524,0.009524
4.0,u1,0.009524,0.009524,0.961905,0.961905
5.0,u1,0.009524,0.009524,0.2,0.009524


,item,posterior
label,,
2.0,u1,1.249217e-05
4.0,u1,1.202314e-05
1.0,u1,1.190410e-07
5.0,u1,4.925712e-08
3.0,u1,1.178623e-09


In [ ]:
prior_df

prior
label item          
1.0   u2    0.214235
      u3    0.214235
2.0   u2    0.214235
      u3    0.214235
3.0   u2    0.214235
      u3    0.214235
4.0   u2    0.214235
      u3    0.214235
5.0   u2     0.14306
      u3     0.14306

In [ ]:
rating_prediction(pos_df)

,item,pred_label
0,u2,1.0
1,u3,3.0
